In [1]:
from samplers import *
from easydict import EasyDict as edict
import numpy as np
import pandas as pd

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def mcmc_sampler(outcome_model,random_state):
    with outcome_model:
        # draw posterior samples
        trace = pm.sample(5000, 
                          tune=5000,
                          cores=1,
                          chains=2,
                          progressbar=False,
                          return_inferencedata=False,
                          random_seed=random_state)
    return trace

In [3]:
def BPH(D,n_intervals=5,
        cov_adj=False,
        random_state=2021):
    
    # preprocessing
    y_interval = D.y.copy()
    y_interval = np.append(y_interval[D.T==1],[np.max(y_interval)+1,0])
    interval_bounds = np.quantile(np.unique(y_interval),np.linspace(0,1,n_intervals+1))
    interval_bounds[1:-1] = np.array([0.95*item if item in interval_bounds else item for item in interval_bounds[1:-1]])
    intervals = np.arange(n_intervals)
    interval_length = np.diff(interval_bounds)
    last_period = np.array([np.sum(interval_bounds<=yy)-1 for yy in D.y]).astype(int)
    n = D.y.shape[0]
    
    # create some datasets
    event_mat = np.zeros((n, n_intervals))
    event_mat[np.arange(n), last_period] = D.delta
    exposure_mat = np.greater_equal.outer(D.y, interval_bounds[:-1]) * interval_length
    exposure_mat[np.arange(n), last_period] = D.y - interval_bounds[last_period]
    
    with pm.Model() as outcome_model:

        lambda0 = pm.Gamma("lambda0", 0.01, 0.01, shape=n_intervals)
        theta1 = pm.Normal("theta1", 0, sigma=100)

        if cov_adj:
            alpha = pm.Normal("alpha", 0, sigma=100, shape=D.X.shape[1])
            lambda_ = pm.Deterministic("lambda_", pm.theano.tensor.outer(pm.math.exp(pm.math.dot(D.X,alpha) + theta1*D.T), 
                                                                     lambda0))
        else:
            lambda_ = pm.Deterministic("lambda_", pm.theano.tensor.outer(pm.math.exp(theta1*D.T),lambda0))
            
        mu = pm.Deterministic("mu", exposure_mat*lambda_)

        obs = pm.Poisson("obs", mu, observed=event_mat)
        
    trace = mcmc_sampler(outcome_model,random_state)
        
    return trace

In [4]:
def BPH_UIP_Dirichlet(D,summaryDs,
                      n_intervals=5,
                      gammas_ps=False,
                      random_state = 2021):
    
    K = len(summaryDs)
    nk_seq = np.array([summaryDs[i].nk for i in range(K)])
    bal_seq = np.array([summaryDs[i].pi_bal for i in range(K)])
    n = D.y.shape[0]

    # gammas: the base measure for the Dirichlet distribution 
    gammas = nk_seq/n
    gammas[gammas>1] = 1

    if gammas_ps:
        pass
#         tmp = bal_seq/bal_seq.min()
#         gammas = gammas * (tmp)

    outcome_model = pm.Model()
    with outcome_model:

        # effective sample size
        M = pm.Uniform('M',lower=0,upper=np.minimum(nk_seq.sum(),n))

        # dirichlet distribution for the prior weighting
        pis = pm.Dirichlet("pis", gammas)

        mu1 = 0 
        sigma1 = 0
        for k in range(K):
            mu1 += pis[k]*summaryDs[k].est
            sigma1 += M*pis[k]/(nk_seq[k]*summaryDs[k].se**2)
        sigma1 = pm.math.sqrt(1/sigma1)

        # preprocessing
        y_interval = D.y.copy()
        y_interval = np.append(y_interval[D.T==1],
                               [np.max(y_interval)+1,0])
        interval_bounds = np.quantile(np.unique(y_interval),np.linspace(0,1,n_intervals+1))
        interval_bounds[1:-1] = np.array([0.95*item if item in interval_bounds else item for item in interval_bounds[1:-1]])
        intervals = np.arange(n_intervals)
        interval_length = np.diff(interval_bounds)
        last_period = np.array([np.sum(interval_bounds<=yy)-1 for yy in D.y]).astype(int)

        # create some datasets
        event_mat = np.zeros((n, n_intervals))
        event_mat[np.arange(n), last_period] = D.delta
        exposure_mat = np.greater_equal.outer(D.y, interval_bounds[:-1]) * interval_length
        exposure_mat[np.arange(n), last_period] = D.y - interval_bounds[last_period]

        lambda0 = pm.Gamma("lambda0", 0.01, 0.01, shape=n_intervals)
        theta1 = pm.Normal('theta1', mu=mu1, sigma=sigma1)

        lambda_ = pm.Deterministic("lambda_", pm.theano.tensor.outer(pm.math.exp(theta1*D.T),lambda0))

        mu = pm.Deterministic("mu", exposure_mat*lambda_)
        obs = pm.Poisson("obs", mu, observed=event_mat)

    trace = mcmc_sampler(outcome_model,random_state)
    
    return trace

In [5]:
NEJM = edict()
NEJM.est = np.log(1.04)
NEJM.se = (np.log(1.32)-np.log(0.82))/(2*1.96)
NEJM.est_hr = 1.04
NEJM.est_ci = (0.82,1.32)
NEJM.nk = 1376
NEJM.name = 'NEJM'

In [6]:
AJE = edict()
AJE.est = np.log(1.21)
AJE.se = (np.log(1.76)-np.log(0.82))/(2*1.96)
AJE.est_hr = 1.21
AJE.est_ci = (0.82,1.76)
AJE.name = 'AJE'
AJE.nk = 770+228

In [8]:
CID = edict()
CID.est = np.log(0.89)
CID.se = (np.log(3.47)-np.log(0.23))/(2*1.96)
CID.est_hr = 0.89
CID.est_ci = (0.23,3.47)
CID.name = 'CID'
CID.nk = 38+46

In [9]:
PLOS_ONE = edict()
PLOS_ONE.est = np.log(1.02)
PLOS_ONE.se = (np.log(1.27)-np.log(0.83))/(2*1.96)
PLOS_ONE.est_hr = 1.02
PLOS_ONE.est_ci = (0.83,1.27)
PLOS_ONE.name = 'PLOS_ONE'
PLOS_ONE.nk = 2512

In [11]:
summaryDs = [PLOS_ONE,NEJM,AJE,CID]

In [12]:
hcq_rct_df = pd.read_csv('./HCQ_RCT.csv').iloc[:,1:]
hcq_rct_df.head()

,time,status,treat
0,1.031941,1,0
1,1.031941,1,0
2,1.916462,1,0
3,2.948403,1,0
4,2.948403,1,0


In [13]:
D = edict()
D.y = hcq_rct_df['time'].values
D.delta = hcq_rct_df['status'].values
D.T = hcq_rct_df['treat'].values

In [14]:
n_intervals = 5
random_state = 2021

In [15]:
trace_nip = BPH(D,n_intervals=n_intervals,
                cov_adj=False,
                random_state=random_state)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [theta1, lambda0]
Sampling 2 chains for 5_000 tune and 5_000 draw iterations (10_000 + 10_000 draws total) took 19 seconds.


In [16]:
trace_uip = BPH_UIP_Dirichlet(D,summaryDs,
                  n_intervals=n_intervals,
                  gammas_ps=False,
                  random_state = random_state)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [theta1, lambda0, pis, M]
Sampling 2 chains for 5_000 tune and 5_000 draw iterations (10_000 + 10_000 draws total) took 39 seconds.


In [17]:
baseline_result = np.array([[item.name, item.nk, item.est_hr, 
           item.est_ci[0], item.est_ci[1]] for i,item in enumerate(summaryDs)])

uip_result = np.array(['RCT-UIP',D.T.shape[0]]+[np.exp(trace_uip['theta1']).mean()]+ \
                        np.percentile(np.exp(trace_uip['theta1']),[2.5,97.5]).tolist())

nip_result = np.array(['RCT-NIP',D.T.shape[0]]+[np.exp(trace_nip['theta1']).mean()]+ \
                        np.percentile(np.exp(trace_nip['theta1']),[2.5,97.5]).tolist())

In [18]:
sum_mat = np.concatenate([baseline_result,
                nip_result.reshape(1,-1),
                uip_result.reshape(1,-1)],axis=0)
sum_mat

array([['PLOS_ONE', '2512', '1.02', '0.83', '1.27'],
       ['NEJM', '1376', '1.04', '0.82', '1.32'],
       ['AJE', '998', '1.21', '0.82', '1.76'],
       ['CID', '84', '0.89', '0.23', '3.47'],
       ['RCT-NIP', '208', '0.9393302718260911', '0.5850004318651876',
        '1.4361737162125814'],
       ['RCT-UIP', '208', '0.96100770923022', '0.6262584431627027',
        '1.3994958258137755']], dtype='<U32')

In [19]:
sum_df = pd.DataFrame(sum_mat)
sum_df.columns = ('Method','Sample Size','HR','CI Lower','CI Upper')
sum_df.iloc[:,3:5] = sum_df.iloc[:,3:5].values.astype(float)
sum_df['CI'] = [(np.round(sum_df['CI Lower'][i],2),np.round(sum_df['CI Upper'][i],2)) for i in range(sum_df.shape[0])]
sum_df['Width'] = sum_df['CI Upper'] - sum_df['CI Lower']
sum_df['Posterior $\pi$'] = trace_uip['pis'].mean(axis=0).tolist()+[np.nan]*2
sum_df['$M$'] = [np.nan]*len(summaryDs)+[np.nan,trace_uip['M'].mean()]
sum_df

,Method,Sample Size,HR,CI Lower,CI Upper,CI,Width,Posterior $\pi$,$M$
0,PLOS_ONE,2512,1.02,0.83,1.27,"(0.83, 1.27)",0.44,0.293783,NaN
1,NEJM,1376,1.04,0.82,1.32,"(0.82, 1.32)",0.5,0.305589,NaN
2,AJE,998,1.21,0.82,1.76,"(0.82, 1.76)",0.94,0.280420,NaN
3,CID,84,0.89,0.23,3.47,"(0.23, 3.47)",3.24,0.120208,NaN
4,RCT-NIP,208,0.9393302718260911,0.585,1.436174,"(0.59, 1.44)",0.851173,NaN,NaN
5,RCT-UIP,208,0.96100770923022,0.626258,1.399496,"(0.63, 1.4)",0.773237,NaN,121.611892
